In [3]:
import pandas as pd
import numpy as np
import openpyxl

In [10]:
# Cross-sectional GSS data processing
# Load GSS overall data
gss = pd.read_excel("data/GSS.xlsx")

In [12]:
gss.shape

(29189, 119)

In [16]:
gss['year'].value_counts()

year
2006    4510
2021    4032
2022    3544
2024    3309
2016    2867
2014    2538
2018    2348
2010    2044
2008    2023
2012    1974
Name: count, dtype: int64

In [37]:
gss['sex']

0        FEMALE
1          MALE
2        FEMALE
3        FEMALE
4          MALE
          ...  
29184      MALE
29185      MALE
29186    FEMALE
29187    FEMALE
29188    FEMALE
Name: sex, Length: 29189, dtype: object

In [44]:
pd.set_option('display.max_columns', None)
gss.head(2)

,year,id_,wrkstat,hrs1,hrs2,wrkslf,occ10,indus10,marital,divorce,sphrs2,spocc10,spind10,sibs,childs,age,educ,speduc,degree,sex,race,reg16,mobile16,family16,famdif16,born,income,rincome,region,partyid,polviews,natenvir,natheal,natcity,natcrime,natdrug,nateduc,natrace,natarms,natfare,natsoc,spkath,libath,spkrac,spkcom,libcom,spkhomo,libhomo,cappun,gunlaw,courts,grass,relig,fund,reliten,postlife,bible,racmar,racseg,racopen,busing,happy,hapmar,health,life,helpful,trust,satjob,richwork,class_,satfin,finalter,finrela,fehome,fework,fepres,abdefect,abnomore,abhlth,abpoor,abrape,absingle,abany,chldidel,premarsx,homosex,pornlaw,letdie1,suicide1,owngun,rowngun,news,tvhours,racdif1,racdif2,racdif3,racdif4,helppoor,helpnot,helpsick,helpblk,givhmlss,volchrty,stress,big5a1,big5b1,big5c1,big5d1,big5e1,big5a2,big5b2,big5c2,big5d2,big5e2,commun,cohort,ballot,wtssps,wtssnrps
0,2006,1,Working full time,35,.i: Inapplicable,Someone else,"Nursing, psychiatric, and home health aides",Nursing care facilities,Never married,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,1,3,50,1 year of college,.i: Inapplicable,High school,FEMALE,Black,Foreign,Different state,Both own parents,.i: Inapplicable,NO,"$25,000 or more",$25000 OR MORE,Middle Atlantic,Strong democrat,Extremely liberal,TOO LITTLE,ABOUT RIGHT,TOO LITTLE,TOO MUCH,TOO MUCH,ABOUT RIGHT,TOO LITTLE,TOO MUCH,TOO MUCH,TOO LITTLE,ALLOWED,NOT REMOVE,NOT ALLOWED,NOT ALLOWED,REMOVE,NOT ALLOWED,NOT REMOVE,FAVOR,OPPOSE,Not harshly enough,Should not be legal,NaN,Liberal,No religion,YES,Word of god,.i: Inapplicable,.i: Inapplicable,Second law,.i: Inapplicable,Pretty happy,.i: Inapplicable,Fair,Routine,Looking out for themselves,Can't be too careful,Very satisfied,Continue to work,Working class,More or less satisfied,Worse,AVERAGE,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,YES,.d: Do not Know/Cannot Choose,YES,NO,NO,NO,YES,.i: Inapplicable,.i: Inapplicable,ALWAYS WRONG,There should be laws against the distribution ...,.i: Inapplicable,.i: Inapplicable,NO,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,GOVT ACTION,Government should do more,GOVT SHOULD HELP,.d: Do not Know/Cannot Choose,.i: Inapplicable,.i: Inapplicable,Never,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,1956,Ballot c,0.505483,0.524713
1,2006,2,Working full time,40,.i: Inapplicable,Someone else,"First-line supervisors of mechanics, installer...","Alcoholic beverages, merchant wholesalers",Never married,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,5,1,27,2 years of college,.i: Inapplicable,High school,MALE,Other,Middle Atlantic,"Same state, same city",Both own parents,.i: Inapplicable,NO,"$25,000 or more",$25000 OR MORE,Middle Atlantic,"Independent, close to republican",Slightly liberal,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,TOO LITTLE,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,FAVOR,.i: Inapplicable,Too harshly,Should not be legal,Catholic,Moderate,Not very strong,.d: Do not Know/Cannot Choose,.d: Do not Know/Cannot Choose,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,Very happy,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,Looking out for themselves,Can't be too careful,Very satisfied,.i: Inapplicable,Working class,More or less satisfied,Better,AVERAGE,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,.i: Inapplicable,3,NOT WRONG AT ALL,.i: Inapplicable,There should be laws against the distribution ...,YES,NO,.i: Inapplicable,.i: Inapplicable,Every day,1,YES,NO,YES,YES,2,Ag

In [60]:
# Create generation column based on birth year
def determine_generation(year):
    if pd.isna(year):
        return None
    elif year.startswith('.'):
        return None
    year = int(year)
    if year <= 1945:
        return "Silent Generation"
    elif 1946 <= year <= 1964:
        return "Baby Boomer"
    elif 1965 <= year <= 1980:
        return "Generation X"
    elif 1981 <= year <= 1996:
        return "Millennial"
    elif 1997 <= year <= 2012:
        return "Generation Z"
    else:
        return "Generation Alpha"

gss['generation'] = gss['cohort'].apply(determine_generation)

In [61]:
# Remove rows with missing values or values starting with "." in key columns
key_columns = ['year', 'partyid', 'sex', 'race', 'generation', 'degree', 'relig', 'polviews']
gss_cleaned = gss.dropna(subset=key_columns)
for col in key_columns:
    gss_cleaned = gss_cleaned[~gss_cleaned[col].astype(str).str.startswith('.')]

In [62]:
gss_cleaned['generation'].value_counts()

generation
Baby Boomer          6817
Generation X         5099
Millennial           3415
Silent Generation    3153
Generation Z          336
Name: count, dtype: int64

In [63]:
# Mapping
gss_maps = {
    'race': {
        "White": "White person",
        "Black": "Black person", 
        "Other": "person of different race than Black or White" 
    },
    'sex': {
        "Male": "man", 
        "Female": "woman"
    },
    'partyid': {
        "Strong democrat" : "Strong Democrat",
        "Not very strong democrat" : "Democrat",
        "Independent, close to democrat" : "Independent leaning Democrat",
        "Independent (neither, no response)" : "Political Independent",
        "Independent, close to republican" : "Independent leaning Republican",
        "Not very strong republican" : "Republican",
        "Strong republican" : "Strong Republican",
        "Other party" : "Other"
    },
    'wrkstat': {
        "Working full time" : "person working full-time",
        "Working part time" : "person working part-time",
        "Retired" : "retired person",
        "In school" : "student",
        "Unemployed, laid off, looking for work" : "unemployed person",
        "Keeping house" : "homemaker",
        "With a job, but not at work because of temporary illness, vacation, strike" : "person with a job but not at work",
        "Other" : "other"
    },
    'degree': {
        "Less than high school" : "person with less than high school education",
        "High school" : "high school graduate",
        "Associate/junior college" : "person with some college education",
        "Bachelor's" : "person with a bachelor's degree",
        "Graduate" : "person with a graduate degree"
    },
    'relig': {
        "Protestant" : "Protestant",
        "Catholic" : "Catholic",
        "Jewish" : "Jewish",
        "Buddhism" : "person of other religion",
        "Muslim/islam" : "person of other religion",
        "Christian" : "person of other religion",
        "Hinduism" : "person of other religion",
        "Orthodox-christian" : "person of other religion",
        "Inter-nondenominational" : "person of other religion",
        "Native american" : "person of other religion",
        "Other eastern religions" : "person of other religion",
        "Other" : "person of other religion"
    },
    'polviews': {
        "Extremely conservative" : "person with an extremely conservative political view",
        "Conservative" : "person with a conservative political view",
        "Slightly conservative" : "person with a slightly conservative political view",
        "Moderate, middle of the road" : "person with a neutral political view",
        "Slightly liberal" : "person with a slightly liberal political view",
        "Liberal" : "person with a liberal political view",
        "Extremely liberal" : "person with an extremely liberal political view"
    },
    'generation': {
        "Silent Generation" : "person from the Silent Generation",
        "Baby Boomer" : "Baby Boomer",
        "Generation X" : "person from Generation X",
        "Millennial" : "Millennial",
        "Generation Z" : "person from Generation Z",
    }
}

In [64]:
# Create a contextualizer

neutral_scenarios = [
    {
        "q": "Describe your morning routine.",
        "a": "I woke up, brushed my teeth, and had a cup of coffee before checking the news."
    },
    {
        "q": "What are you having for lunch?",
        "a": "I'm planning to have a sandwich and a salad at a local cafe."
    },
    {
        "q": "How is the weather today?",
        "a": "It is partly cloudy and a bit brisk, but the sun is peeking through."
    },
    {
        "q": "What are your plans for the weekend?",
        "a": "I plan to relax at home, catch up on some reading, and maybe go for a walk."
    },
    {
        "q": "Describe your commute.",
        "a": "I traveled to the city center and listened to a podcast on the way."
    }
]

In [65]:
# Create a generator function

def create_contrastive_dataset(demographic_label, baseline_label="person"):

    """
    Generates a list of (X+ and X-) pairs for a specific trait.
    """

    dataset = []

    for scen in neutral_scenarios:
        # X+ example: "[INST] You are a Black person. Describe your morning routine. [/INST] I woke up..."
        pos_prompt = f"[INST] You are a (an) {demographic_label} living in the United States. {scen['q']} [/INST] {scen['a']}"

        # X- example: "[INST] You are a person. Describe your morning routine. [/INST] I woke up..."
        neg_prompt = f"[INST] You are a (an) {baseline_label} living in the United States. {scen['q']} [/INST] {scen['a']}"

        dataset.append((pos_prompt, neg_prompt))
    
    return dataset

In [66]:
# Generate prompts for unique groups
extraction_datasets = {}

# Race
unique_races = gss_cleaned['race'].unique()
for r_code in unique_races:
    trait_text = gss_maps['race'].get(r_code, "person")

    # Generate the dataset for this specific race
    pairs = create_contrastive_dataset(trait_text, baseline_label="person")
    extraction_datasets[f"Race_{trait_text}"] = pairs

# Party ID
unique_parties = gss_cleaned['partyid'].unique()
for p_code in unique_parties:
    trait_text = gss_maps['partyid'].get(p_code, "voter")

    # Generate the dataset for this specific party ID
    pairs = create_contrastive_dataset(trait_text, baseline_label="person")
    extraction_datasets[f"PartyID_{trait_text}"] = pairs

# Sex
unique_sexes = gss_cleaned['sex'].unique()
for s_code in unique_sexes:
    trait_text = gss_maps['sex'].get(s_code, "person")

    # Generate the dataset for this specific sex
    pairs = create_contrastive_dataset(trait_text, baseline_label="person")
    extraction_datasets[f"Sex_{trait_text}"] = pairs

# Degree
unique_degrees = gss_cleaned['degree'].unique()
for d_code in unique_degrees:
    trait_text = gss_maps['degree'].get(d_code, "person")

    # Generate the dataset for this specific degree
    pairs = create_contrastive_dataset(trait_text, baseline_label="person")
    extraction_datasets[f"Degree_{trait_text}"] = pairs

# Religion
unique_religions = gss_cleaned['relig'].unique()
for r_code in unique_religions:
    trait_text = gss_maps['relig'].get(r_code, "person")

    # Generate the dataset for this specific religion
    pairs = create_contrastive_dataset(trait_text, baseline_label="person")
    extraction_datasets[f"Religion_{trait_text}"] = pairs

# Political Views
unique_pviews = gss_cleaned['polviews'].unique()
for pv_code in unique_pviews:
    trait_text = gss_maps['polviews'].get(pv_code, "person")

    # Generate the dataset for this specific political view
    pairs = create_contrastive_dataset(trait_text, baseline_label="person")
    extraction_datasets[f"PolViews_{trait_text}"] = pairs

# Generation
unique_genviews = gss_cleaned['generation'].unique()
for gen_code in unique_genviews:
    trait_text = gss_maps['generation'].get(gen_code, "person")

    # Generate the dataset for this specific generation
    pairs = create_contrastive_dataset(trait_text, baseline_label="person")
    extraction_datasets[f"Generation_{trait_text}"] = pairs

In [72]:
len(extraction_datasets)

33

In [74]:
target_key = "Race_Black person"
if target_key in extraction_datasets:
    print(f"--- Prompts generated for '{target_key}' ---")
    for i, (pos, neg) in enumerate(extraction_datasets[target_key]):
        print(f"\nPair {i+1}:")
        print(f"  X+ (Target): {pos}")
        print(f"  X- (Generic):{neg}")

--- Prompts generated for 'Race_Black person' ---

Pair 1:
  X+ (Target): [INST] You are a (an) Black person living in the United States. Describe your morning routine. [/INST] I woke up, brushed my teeth, and had a cup of coffee before checking the news.
  X- (Generic):[INST] You are a (an) person living in the United States. Describe your morning routine. [/INST] I woke up, brushed my teeth, and had a cup of coffee before checking the news.

Pair 2:
  X+ (Target): [INST] You are a (an) Black person living in the United States. What are you having for lunch? [/INST] I'm planning to have a sandwich and a salad at a local cafe.
  X- (Generic):[INST] You are a (an) person living in the United States. What are you having for lunch? [/INST] I'm planning to have a sandwich and a salad at a local cafe.

Pair 3:
  X+ (Target): [INST] You are a (an) Black person living in the United States. How is the weather today? [/INST] It is partly cloudy and a bit brisk, but the sun is peeking through.
 

In [76]:
# Save the extraction datasets to a JSON file
import json
with open("data/gss_extraction_datasets.json", "w") as f:
    json.dump(extraction_datasets, f, indent=4)